In [66]:
from PIL import Image
from queue import Queue
import numpy as np

In [67]:
def distance(x, y):
    return (y[0]-x[0])**2 + (y[1]-x[1])**2 + (y[2]-x[2])**2

In [68]:
im_bg = Image.open("images/bg.png")
im_1 = Image.open("images/2.png")

(left, upper, right, lower) = (503, 263, 1436, 823)
im_c_bg = im_bg.crop((left, upper, right, lower))
im_c_1 = im_1.crop((left, upper, right, lower))

In [69]:
width = right - left      # 933
height = lower - upper    # 560

im_sub = Image.new(mode='1', size=(width, height))

for y in range(height):
    for x in range(width):
        p1 = im_c_bg.getpixel((x, y))
        p2 = im_c_1.getpixel((x, y))
        if(distance(p1, p2) > 800):
            im_sub.putpixel((x, y), 1)
            
im_sub.show()

In [84]:
class Pixel:
    def __init__(self, x, y):
        self.x = x
        self.y = y

In [83]:
def get_neighbors(p, width, height):
    neighbors = []
    for i in range(-1, 2):
        for j in range(-1, 2):
            if(not(i==0 and j==0)):
                xn = p.x + i
                yn = p.y + j
                if(xn>=0 and xn<width and yn>=0 and yn<height):
                    neighbors.append(Pixel(xn, yn))
                    
    return neighbors

In [82]:
def label_neighbors(fg_chk, q, width, height, labels):
    while(not q.empty()):
        p = q.get()
        for pn in get_neighbors(p, width, height):
            if(fg_chk[pn.y, pn.x]==1 and labels[pn.y, pn.x]==-1):
                labels[pn.y, pn.x] = current_label
                q.put(pn)
                
    return labels

In [85]:
fg_chk = np.full((height, width), -1)
labels = np.full((height, 
                  width), -1)

In [86]:
for y in range(height):
    for x in range(width):
        fg_chk[y, x] = im_sub.getpixel((x,y))

In [87]:
current_label = 1
q = Queue()

for y in range(height):
    for x in range(width):
        if(fg_chk[y, x]==1 and labels[y, x]==-1):
            labels[y, x] = current_label
            q.put(Pixel(x, y))
            labels = label_neighbors(fg_chk, q, width, height, labels)
            current_label += 1

In [90]:
print(current_label)

3


In [91]:
im_label = Image.new(mode='1', size=(width, height))
im_arr = []

for i in range(current_label):
    x_min = width
    x_max = 0
    y_min = height
    y_max = 0
    
    for y in range(height):
        for x in range(width):
            if(labels[y, x]==i):
                if(x < x_min):
                    x_min = x
                if(x > x_max):
                    x_max = x
                if(y < y_min):
                    y_min = y
                if(y > y_max):
                    y_max = y
    
    im_arr.append(im_c_1.crop((x_min, y_min, x_max, y_max)))

In [93]:
len(im_arr)

3

SystemError: tile cannot extend outside image